In [1]:
import dimod
import dwave.inspector
from dwave.system import DWaveSampler, EmbeddingComposite

In [2]:
def generateQUBO(n, penalties=None):
    '''
    Generate the QUBO for a Simon's problem. The "secret string" is the
    bitstring of all 1s.
    PARAMS:
        n       (int):   The number of qubits in the oracle.
        penalty (array): The penalties added to output transitions.
    RETURNS:
        (dict) The QUBO for the Simon's problem.
    '''

    def indexIter():
        '''
        An iterator to track qubit indices.
        '''
        i = 0
        while True:
            yield i
            i += 1

    # No penalties added by default.
    if type(penalties) == None:
        penalties = [0] * (n - 1)

    # Initialize needed variables.
    Q = dict()
    qubitIndex = indexIter()
    in2 = next(qubitIndex)
    Q[(in2, in2)] = 0.0

    # For each qubit:
    for i in range(n-1):

        # Update qubit indices.
        in1 = in2
        in2 = next(qubitIndex)
        out = next(qubitIndex)
        ancilla = next(qubitIndex)

        # Add a one qubit to the QUBO oracle.
        Q[(in1, in1)] += 1.0
        Q[(in1, in2)] = 2.0
        Q[(in1, out)] = -2.0
        Q[(in1, ancilla)] = -4.0
        Q[(in2, in2)] = 1.0
        Q[(in2, out)] = -2.0
        Q[(in2, ancilla)] = -4.0
        Q[(out, out)] = 1.0 + penalties[i]
        Q[(out, ancilla)] = 4.0
        Q[(ancilla, ancilla)] = 4.0

    return Q

In [6]:
N = 50

# Get sampler
sampler = EmbeddingComposite(
    DWaveSampler(
        solver="Advantage_system5.4",
        token="julr-a86ece088ec3ae431ae7ee0541c03112c43d7af4",
        region="eu-central-1",
    )
)

qubo = generateQUBO(N, [2*(-1**i) for i in range(N-1)])
# Define a problem
params = list(dimod.Binaries([f'x{n}' for n in range(3*N-2)]))

bqm = sum([coeff*params[x1]*params[x2] for ((x1, x2), coeff) in list(qubo.items())])

# Sample
sampleset = sampler.sample(bqm, num_reads=100)

# Inspect
dwave.inspector.show(sampleset)

Serving Inspector on http://127.0.0.1:18002/?problemId=28d2e925-0ac1-4be2-b93c-93b01a86cb73

'http://127.0.0.1:18002/?problemId=28d2e925-0ac1-4be2-b93c-93b01a86cb73'

In [4]:
N = 3

# Get sampler
sampler = EmbeddingComposite(
    DWaveSampler(
        solver="Advantage_system5.4",
        token="julr-a86ece088ec3ae431ae7ee0541c03112c43d7af4",
        region="eu-central-1",
    )
)

qubo = generateQUBO(N, [2*(-1**i) for i in range(N-1)])
# Define a problem
params = list(dimod.Binaries([f'x{n}' for n in range(3*N-2)]))

bqm = sum([coeff*params[x1]*params[x2] for ((x1, x2), coeff) in list(qubo.items())])

# Sample
sampleset = sampler.sample(bqm, num_reads=100)

# Inspect
dwave.inspector.show(sampleset)


Serving Inspector on http://127.0.0.1:18002/?problemId=b4669f63-e72b-4cb2-8d46-154243ef2fb7

'http://127.0.0.1:18002/?problemId=b4669f63-e72b-4cb2-8d46-154243ef2fb7'